In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import json
import time
import itertools
from scipy import stats
import psycopg2 as psql
from psycopg2.extras import RealDictCursor

import seaborn as sns
sns.set(color_codes=True)

/home/victor/.local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [31]:
def create_global_box(f1, f2):
    f = f1 + f2
    fuz = list(zip(*f))

    return [np.nanmin(fuz[0]), np.nanmax(fuz[0]), np.nanmin(fuz[1]), np.nanmax(fuz[1])]


def part_gl_box(box):
    latd = 0.167
    lond = 0.271

    lat_lst = [box[0] + i * latd for i in range(int(math.floor((box[1] - box[0]) / latd)))]
    lat_lst.append(box[1])
    lon_lst = [box[2] + i * lond for i in range(int(math.floor((box[3] - box[2]) / lond)))]
    lon_lst.append(box[3])

    boxes=[]

    for i in range(len(lat_lst) - 1):
        for ii in range(len(lon_lst) - 1):
            boxes.append((lat_lst[i], lat_lst[i + 1], lon_lst[ii], lon_lst[ii + 1]))

    return boxes


def check_box(f, b):
    for lat, lon, ts in f:
        if (b[0] <= lat <= b[1]) & (b[2] <= lon <= b[3]):
            return True

    return False


def find_fl_boxes(f1, f2, boxes):
    box_lst = []

    for box in boxes:
        if check_box(f1, box):
            if check_box(f2, box):
                box_lst.append(box)

    return list(set(box_lst))


def resample_flight(box, f):
    """Flights should be zipped list like zip(lat,lon)"""

    f_res = [(lat, lon, ts) for lat, lon, ts in f if (box[0] <= lat <= box[1]) & (box[2] <= lon <= box[3])]

    return f_res


def calc_coord_dst_simple(c1, c2):
    R = 6371.1 * 1000  # Radius of the Earth in m

    lon1 = c1[0]
    lat1 = c1[1]
    lon2 = c2[0]
    lat2 = c2[1]

    [lon1, lat1, lon2, lat2] = [math.radians(l) for l in [lon1, lat1, lon2, lat2]]

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    x = dlon * math.cos(dlat / 2)
    y = dlat
    d = math.sqrt(x * x + y * y) * R

    return d


def calc_coord_dst(c1, c2):
    R = 6371.1 * 1000  # Radius of the Earth in m

    lat1 = c1[0]
    lon1 = c1[1]
    lat2 = c2[0]
    lon2 = c2[1]

    [lon1, lat1, lon2, lat2] = [math.radians(l) for l in [lon1, lat1, lon2, lat2]]

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    d = R * c
    return d


def calc_bearing(c0, c1):
    if not all(isinstance(i, tuple) for i in [c0, c1]):
        return np.nan

    lat1 = c0[0]
    lon1 = c0[1]
    lat2 = c1[0]
    lon2 = c1[1]

    [lon1, lat1, lon2, lat2] = [math.radians(l) for l in [lon1, lat1, lon2, lat2]]

    dlon = lon2 - lon1

    x = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(dlon)
    y = math.sin(dlon) * math.cos(lat2)
    bearing = math.atan2(y, x)

    return math.degrees(bearing)


def box_area(box):
    w = calc_coord_dst_simple([box[0], box[2]], [box[1], box[2]])
    h = calc_coord_dst_simple([box[0], box[2]], [box[0], box[3]])

    return w * h


def closest_distance(f1, f2):
    """Flights should be zipped list like zip(lat,lon)"""

    x = [[np.sqrt((c1[0] - c2[0]) ** 2 + (c1[1] - c2[1]) ** 2) for c1 in f1] for c2 in f2]
    dmin = np.nanmin(x)
    if2, if1 = np.where(x == dmin)

    c2 = f2[if2[0]]
    c1 = f1[if1[0]]
    dmin_m = calc_coord_dst(c1, c2)

    return dmin_m, c1, c2, if1[0], if2[0]


def dst_aligned_times(f1, f2):
    """Flights should be zipped list like zip(lat,lon,ts)"""

    t = [[abs(c1[2] - c2[2]) for c1 in f1] for c2 in f2]
    tmin = np.nanmin(t)
    if2, if1 = np.where(t == tmin)


def find_flight_intersect(f1, f2):
    gl_box = create_global_box(f1, f2)
    box_lst = part_gl_box(gl_box)
    fl_boxes = find_fl_boxes(f1, f2, box_lst)

    confl_list = []

    if not fl_boxes:
        return None

    for b in fl_boxes:
        f1r = resample_flight(b, f1)
        f2r = resample_flight(b, f2)

        dminr, c1r, c2r, if1r, if2r = closest_distance(f1r, f2r)
        confl_list.append((dminr, c1r, c2r, if1r, if2r))

    return confl_list

In [34]:
    try:
        conn = psql.connect("dbname='thesisdata' user='postgres' host='localhost' password='postgres'")
    except Exception as e:
        print("Unable to connect to the database.")
        print(e)

    fl_start_ep = 1512225284
    fetch_batch_size = 500
    ts_offset = 3600
    max_dst = 5 * 1852
    max_ts = 30
    alt_min = 15000

    cur_read = conn.cursor(cursor_factory=RealDictCursor)
    cur_read.execute("SELECT ts, lat, lon, alt, spd, hdg, roc, start_ep, flight_id \
                     FROM public.adsb_flights WHERE flight_length > 500 \
                     AND start_ep BETWEEN %s AND %s;",
                     (fl_start_ep - ts_offset, fl_start_ep + ts_offset))

    f_list = []
    fsets = []
    sql_inj_lst = []
    col_lst = ['td', 'altd', 'dstd', 'hdgd',
               'flight_id_1', 'ts_1', 'lat_1', 'lon_1', 'alt_1', 'spd_1', 'hdg_1', 'roc_1',
               'flight_id_2', 'ts_2', 'lat_2', 'lon_2', 'alt_2', 'spd_2', 'hdg_2', 'roc_2']

    batch = cur_read.fetchall()

    id_lst = [b['flight_id'] for b in batch]
    fset = list(itertools.combinations(id_lst, 2))

    cnt = 1
    print(len(fset))

    for fs in fset:

        if (not cnt % 1000):
            print(cnt)

        f1 = next(f for f in batch if f['flight_id'] == fs[0])
        f2 = next(f for f in batch if f['flight_id'] == fs[1])


        if f1['flight_id'] == f2['flight_id']:
            continue

        if abs(f1['start_ep'] - f2['start_ep']) < 1800:
            # f1 = {k: v for k, v in f1.items() if }
            f1crd = [(lt, ln, ts) for lt, ln, alt, ts in
                     list(zip(f1['lat'], f1['lon'], f1['alt'], f1['ts'])) if alt > alt_min]

            f2crd = [(lt, ln, ts) for lt, ln, alt, ts in
                     list(zip(f2['lat'], f2['lon'], f2['alt'], f2['ts'])) if alt > alt_min]

            if len(f2crd) > 20 and len(f1crd) > 20:

                t1 = time.time()

                confl_lst = find_flight_intersect(f1crd, f2crd)
                
                print("calc took: %f" % (time.time() - t1))
                if confl_lst:

#                     plt.figure(figsize=(30, 30))

#                     plt.scatter([cx[0] for cx in f1crd], [cx[1] for cx in f1crd], c='b')
#                     plt.scatter([cx[0] for cx in f2crd], [cx[1] for cx in f2crd], c='r')
#                     plt.show()

                    print(confl_lst)

#                     break

68265
calc took: 0.066830
calc took: 0.087276
calc took: 0.067824
calc took: 0.107347
calc took: 0.093861
[(9009.839291981496, (51.40654, 0.24994, 1512228385.63), (51.32556, 0.24556, 1512225125.1), 4, 2)]
calc took: 0.090190
calc took: 0.065266
calc took: 0.113317
calc took: 0.062873
calc took: 0.090963
calc took: 0.119657
calc took: 0.150977
calc took: 0.101516
calc took: 0.097095
[(13376.867238656529, (51.57834, 0.87898, 1512228199.9), (51.69658, 0.84327, 1512227261.34), 10, 0), (14322.049296853558, (51.59571, 1.25671, 1512228089.75), (51.7245, 1.25422, 1512227090.57), 40, 19), (13381.820305132078, (51.58818, 1.78379, 1512227934.19), (51.7085, 1.78764, 1512226895.64), 1, 0), (5683.5501926203215, (51.60544, 2.05326, 1512227858.63), (51.65428, 2.02898, 1512226807.77), 2, 0), (12311.468097620225, (51.59839, 1.01995, 1512228159.25), (51.70908, 1.02396, 1512227184.76), 24, 26), (119.2685241256705, (51.61954, 2.16809, 1512227825.44), (51.62061, 2.16797, 1512226759.68), 17, 36)]
calc took: 

calc took: 0.081845
calc took: 0.142704
calc took: 0.091517
[(1628.5363443804206, (51.53055, 0.70639, 1512228250.85), (51.543, 0.69399, 1512224939.2), 0, 24), (226.26357710360992, (51.53233, 0.71292, 1512228248.87), (51.53188, 0.70973, 1512224947.21), 18, 0)]
calc took: 0.096924
calc took: 0.051419
calc took: 0.057939
calc took: 0.095883
calc took: 0.074514
calc took: 0.051606
[(22421.66409145533, (51.9147, 3.13972, 1512227513.05), (52.08134, 2.95532, 1512228261.85), 0, 20)]
calc took: 0.085949
[(1372.0593508352792, (51.60911, 2.08282, 1512227849.72), (51.59678, 2.08206, 1512229233.88), 29, 56), (4341.517623018251, (51.58818, 1.78379, 1512227934.19), (51.62718, 1.78676, 1512229334.87), 1, 0), (461.47445773562663, (51.60544, 2.05326, 1512227858.63), (51.60129, 2.05322, 1512229243.82), 2, 1)]
calc took: 0.114589
calc took: 0.060155
calc took: 0.066513
[(292.04609069308515, (51.5983, 1.00221, 1512228164.21), (51.60064, 1.00029, 1512227449.46), 28, 1), (3626.752752171712, (51.59627, 0.9685

calc took: 0.106634
calc took: 0.087613
[(6307.237436089685, (51.59641, 1.19965, 1512228106.69), (51.64362, 1.25029, 1512224478.99), 0, 13), (585.0743335676069, (51.59496, 1.31073, 1512228073.57), (51.60022, 1.31094, 1512224509.87), 31, 7)]
calc took: 0.055796
calc took: 0.093029
calc took: 0.090692
calc took: 0.130366
[(11415.305835139108, (51.22929, -0.38963, 1512228570.69), (51.14099, -0.30609, 1512228510.57), 0, 3), (21938.204060278702, (51.43319, 0.3479, 1512228356.33), (51.59869, 0.17532, 1512228812.59), 3, 22), (280.42615816184036, (51.29945, -0.13885, 1512228499.57), (51.298, -0.14215, 1512228614.68), 11, 0)]
calc took: 0.103556
calc took: 0.141576
calc took: 0.046978
[(14994.920263033724, (51.9147, 3.13972, 1512227513.05), (52.04955, 3.14028, 1512225640.54), 0, 99), (8837.168629847658, (51.96766, 3.36613, 1512227437.53), (52.04713, 3.36731, 1512225733.45), 0, 83)]
calc took: 0.046214
[(11998.84454453909, (51.59627, 0.96855, 1512228173.71), (51.70417, 0.96664, 1512225607.39), 0

KeyboardInterrupt: 

In [45]:
x = np.array([1,3,5,7,9])
np.where(x == 7)

(array([3]),)

In [46]:
conn.close()